# Import libraries

In [204]:
import gym
import numpy as np
import time
from IPython import display
from prettytable import PrettyTable

# Function

In [205]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [206]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')
    return success/max_episodes

In [207]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    return policy

In [208]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

# Value iteration

In [209]:
runtimes_ValueIter = []
winrates_ValueIter = []

In [210]:
def value_iteration(env, max_iters=500, gamma=0.9):
  # initialize
  v_values = np.zeros(env.observation_space.n)

  for i in range(max_iters):
    prev_v_values = np.copy(v_values)

      # update the v-value for each state
    for state in range(env.observation_space.n):
      q_values = []
      
      # compute the q-value for each action that we can perform at the state
      for action in range(env.action_space.n):
        q_value = 0
        # loop through each possible outcome
        for prob, next_state, reward, done in env.P[state][action]:
            q_value += prob * (reward + gamma * prev_v_values[next_state])
        
        q_values.append(q_value)
      
        # select the max q-values
        best_action = np.argmax(q_values)
        v_values[state] = q_values[best_action]

    # check convergence
    if np.all(np.isclose(v_values, prev_v_values)):
      print(f'Converged at {i}-th iteration.')
      break
    
  return v_values

'FrozenLake-v1'

In [211]:
env = gym.make('FrozenLake-v1')
print("observation_space: ",env.observation_space.n)
print("action_space: ",env.action_space.n)
print('----------------------------')
start = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
end = time.time()

optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)
winrate = play_multiple_times(env, optimal_policy, 1000)

runtimes_ValueIter.append(end-start)
winrates_ValueIter.append(winrate)
print("Runtime: ", end-start)

observation_space:  16
action_space:  4
----------------------------
Converged at 79-th iteration.
Number of successes: 720/1000
Average number of steps: 36.423611111111114
Runtime:  0.11992216110229492


'FrozenLake8x8-v0'

In [212]:
env = gym.make('FrozenLake8x8-v1')
print("observation_space: ",env.observation_space.n)
print("action_space: ",env.action_space.n)
print('----------------------------')
start = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
end = time.time()

optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)
winrate = play_multiple_times(env, optimal_policy, 1000)

runtimes_ValueIter.append(end-start)
winrates_ValueIter.append(winrate)
print("Runtime: ", end-start)

observation_space:  64
action_space:  4
----------------------------
Converged at 117-th iteration.
Number of successes: 732/1000
Average number of steps: 73.73224043715847
Runtime:  0.7215578556060791


'Taxi-v3'

In [213]:
env = gym.make('Taxi-v3')
print("observation_space: ",env.observation_space.n)
print("action_space: ",env.action_space.n)
print('----------------------------')
start = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
end = time.time()

optimalPolicy = policy_extraction(env, optimal_v_values, gamma=0.9)
winrate = play_multiple_times(env, optimalPolicy, 1000)

runtimes_ValueIter.append(end-start)
winrates_ValueIter.append(winrate)
print("Runtime: ", end-start)

observation_space:  500
action_space:  6
----------------------------
Converged at 116-th iteration.
Number of successes: 1000/1000
Average number of steps: 12.989
Runtime:  5.267210006713867


# Policy Iteration

In [214]:
def policy_iteration(env, max_iters=500, gamma=0.9):
    policy = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_policy = np.copy(policy)

        # Policy Evaluation
        v_values = policy_evaluation(env, policy, max_iters=max_iters, gamma=gamma)

        # Policy Improvement
        policy = policy_extraction(env, v_values, gamma=gamma)

        # Check convergence
        is_converged = True
        for state in range(env.observation_space.n):
            if policy[state] != prev_policy[state]:
                is_converged = False
                break

        if is_converged:
            print(f'Converged at {i}-th iteration.')
            break
    return policy

In [215]:
runtimes_PolicyIter = []
winrates_PolicyIter = []

'FrozenLake-v1'

In [216]:
# Create the environment
env = gym.make('FrozenLake-v1')
print("observation_space:", env.observation_space.n)
print("action_space:", env.action_space.n)
print('----------------------------')

# Perform policy iteration
start = time.time()
optimalPolicy = policy_iteration(env, max_iters=500, gamma=0.9)
end = time.time()
print("Runtime:", end - start)

# Play multiple times and calculate win rate
winrate = play_multiple_times(env, optimalPolicy, 1000)

runtimes_PolicyIter.append(end - start)
winrates_PolicyIter.append(winrate)

observation_space: 16
action_space: 4
----------------------------
Converged at 0-th iteration.
Converged at 23-th iteration.
Converged at 59-th iteration.
Converged at 62-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Converged at 5-th iteration.
Runtime: 0.07497310638427734
Number of successes: 730/1000
Average number of steps: 37.487671232876714


'FrozenLake8x8-v1'

In [217]:
# Create the environment
env = gym.make('FrozenLake8x8-v1')
print("observation_space:", env.observation_space.n)
print("action_space:", env.action_space.n)
print('----------------------------')

# Perform policy iteration
start = time.time()
optimalPolicy = policy_iteration(env, max_iters=500, gamma=0.9)
end = time.time()
print("Runtime:", end - start)

# Play multiple times and calculate win rate
winrate = play_multiple_times(env, optimalPolicy, 1000)
runtimes_PolicyIter.append(end - start)
winrates_PolicyIter.append(winrate)

observation_space: 64
action_space: 4
----------------------------
Converged at 27-th iteration.
Converged at 91-th iteration.
Converged at 92-th iteration.
Converged at 86-th iteration.
Converged at 90-th iteration.
Converged at 92-th iteration.
Converged at 95-th iteration.
Converged at 100-th iteration.
Converged at 112-th iteration.
Converged at 117-th iteration.
Converged at 9-th iteration.
Runtime: 0.4588966369628906
Number of successes: 741/1000
Average number of steps: 70.26585695006747


'Taxi-v3'

In [218]:
# Create the environment
env = gym.make('Taxi-v3')
print("observation_space:", env.observation_space.n)
print("action_space:", env.action_space.n)
print('----------------------------')

# Perform policy iteration
start = time.time()
optimalPolicy = policy_iteration(env, max_iters=500, gamma=0.9)
end = time.time()
print("Runtime:", end - start)

# Play multiple times and calculate win rate
winrate = play_multiple_times(env, optimalPolicy, 1000)

runtimes_PolicyIter.append(end - start)
winrates_PolicyIter.append(winrate)

observation_space: 500
action_space: 6
----------------------------
Converged at 88-th iteration.
Converged at 97-th iteration.
Converged at 100-th iteration.
Converged at 101-th iteration.
Converged at 102-th iteration.
Converged at 103-th iteration.
Converged at 106-th iteration.
Converged at 109-th iteration.
Converged at 110-th iteration.
Converged at 111-th iteration.
Converged at 112-th iteration.
Converged at 115-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 16-th iteration.
Runtime: 6.607874393463135
Number of successes: 1000/1000
Average number of steps: 12.971


#Conclution

In [219]:
#runtime
table = PrettyTable()
table.field_names = ["Algorithms", "FrozenLake-v1", "'FrozenLake8x8-v1", "Taxi-v3"]
table.add_row(["Value Iteration", runtimes_ValueIter[0], runtimes_ValueIter[1], runtimes_ValueIter[2]])
table.add_row(["Policy Iteration", runtimes_PolicyIter[0], runtimes_PolicyIter[1], runtimes_PolicyIter[2]])

title = "RUNTIME"
title_width = len(table.get_string(fields=["Algorithms", "FrozenLake-v1", "'FrozenLake8x8-v1", "Taxi-v3"]).splitlines()[0])
print(f"{title:^{title_width}}")
print(table)
# winrate
table = PrettyTable()
table.field_names = ["Algorithms", "FrozenLake-v1", "'FrozenLake8x8-v1", "Taxi-v3"]
table.add_row(["Value Iteration", winrates_ValueIter[0], winrates_ValueIter[1], winrates_ValueIter[2]])
table.add_row(["Policy Iteration", winrates_PolicyIter[0], winrates_PolicyIter[1], winrates_PolicyIter[2]])

title = "WINRATE"
title_width = len(table.get_string(fields=["Algorithms", "FrozenLake-v1", "'FrozenLake8x8-v1", "Taxi-v3"]).splitlines()[0])
print(f"{title:^{title_width}}")
print(table)

                                      RUNTIME                                      
+------------------+---------------------+--------------------+-------------------+
|    Algorithms    |    FrozenLake-v1    | 'FrozenLake8x8-v1  |      Taxi-v3      |
+------------------+---------------------+--------------------+-------------------+
| Value Iteration  | 0.11992216110229492 | 0.7215578556060791 | 5.267210006713867 |
| Policy Iteration | 0.07497310638427734 | 0.4588966369628906 | 6.607874393463135 |
+------------------+---------------------+--------------------+-------------------+
                             WINRATE                              
+------------------+---------------+-------------------+---------+
|    Algorithms    | FrozenLake-v1 | 'FrozenLake8x8-v1 | Taxi-v3 |
+------------------+---------------+-------------------+---------+
| Value Iteration  |      0.72     |       0.732       |   1.0   |
| Policy Iteration |      0.73     |       0.741       |   1.0   |
+---------

**Comment**
 
*   Winrate của 2 algorithms trong 3 toy games đều gần giống nhau
*   Runtime của 2 algorithms trong 3 toy games cũng gần tương đương nhau(có sự chênh lệnh tùy vào mỗi toy game)